# DRAFT -- Benjamin Franklin's Wikipedia Social Network -- First Attempt

## This is a working draft of my first attempt (scraping information, organizing graph, creating visuals, etc.). I decided rather than use a rule based approach to identifying people from their text, I'd rather use a machine learning text classifier, so I adjusted my approach to gathering the data. Please see 

Basically, scrape all PEOPLE links from Ben Franklin's wikipedia page and scrape PEOPLE links on Ben Franklins page to get the directed network (who links to whom). Network analysis, Influencers, etc.

In [1]:
from bs4 import BeautifulSoup as bs
import requests
import nltk
import os
import spacy
import re
import time

import pandas as pd
import numpy as np
import networkx as nx

nlp = spacy.load('xx_ent_wiki_sm')
path_to_gs = "C:\\Program Files\\gs\\gs9.23\\bin"
os.environ['PATH'] += os.pathsep + path_to_gs

In [ ]:
#nltk.download('maxent_ne_chunker')
#nltk.download('words')

In [2]:
BenFrank = requests.get('https://en.wikipedia.org/wiki/Benjamin_Franklin')
source_B = bs(BenFrank.text, 'html.parser')
links_B = source_B.find_all('a', href = True)

In [253]:
api_call = "http://en.wikipedia.org/w/api.php?action=parse&format=json&page="
    
page = requests.get(api_call + 'Benjamin Franklin')
source = bs(page.json()['parse']['text']['*'], 'html.parser')

# Get all paragraph text
p = source.find_all('p')

links_p = []
for para in p:
    p_source = p_source = bs(str(para), 'html.parser')
    lis = p_source.find_all('a', href = True)
    for lin in lis:
        links_p.append(lin)

In [3]:
names = []
front = 'https://en.wikipedia.org'

names = []
for i in range(len(links_p)):
    title = links_p[i].get('title')
    try:
        doc = nlp(title)
        if doc.ents[0].label_ == 'PER':
            names.append((links_p[i].get('title'), front + links_p[i].get('href')))
    except:
        continue
unq_names = set(names)

In [252]:
len(unq_names) # Should be 156

156

In [249]:
df_links = pd.DataFrame(columns=['Person', 'Links to Person'])

In [ ]:
approved = []
rejected = []
links_db = {}
person_links_db = {}
all_checked = []
api_call = "http://en.wikipedia.org/w/api.php?action=parse&format=json&page="

all_info = {'app': approved, 'rej':rejected, 'ldb':links_db, 'pldb':person_links_db, 'all':all_checked, 'api':api_call}


def parse_page(page_title, all_info):
        # Get page and parse
        page = requests.get(all_info['api'] + page_title)
        source = bs(page.json()['parse']['text']['*'], 'html.parser')

        check_person = check_if_person(source, page_title)

        if check_person is False:
            all_info['rej'].append(page_title)
            all_info['all'].append(page_title)
            return all_info

        elif check_person is True:
            all_info['app'].append(page_title)
            all_info['all'].append(page_title)

            # Now we know if this page is a person
            
            # Get all links
            links = []
            p = source.find_all('p')

            for para in p:
                p_source = bs(str(para), 'html.parser')
                links_p = p_source.find_all('a', href = True)
                for link in links_p:
                    
                    #Check if this link has been checked already
                    title = link.get('title')
                    
                    if title not in all_info['all']:
                        
                        # Not checked already, check if person
                        check_person = check_if_person(source, title)

                        if check_person is False:
                            all_info['rej'].append(title)
                            all_info['all'].append(title)

                        elif check_person is True:
                            all_info['app'].append(title)
                            all_info['all'].append(title)
                            
                            links.append(title)
                    # If already checked, check if approved
                    elif title in all_info['app']:    
                        links.append(title)
            all_info['ldb'][page_title] = links

            return all_info    
        
    
        
    

In [ ]:
def check_if_person(source, name):
    
    # Get all paragraph text
    p = source.find_all('p')
    
    # Check if there are year(s) in the first 2 p (birth/death date)
    temp = []
    for i in [0,1]:
        temp = re.findall(pattern= r'([\d]{4})', string=str(p[i]))
        if len(temp) > 0: 
            break

    # Check if there is a mention of a relationship in the first 6 paragraphs
    relation = []
    range_look = range(6)
    if len(p) < 6:
        range_look = range(len(p))
    for i in range_look:
        relation = re.findall(pattern= r'((?:Born|born|Father|father|mother|Mother|Son|son|Daugher|daughter|child))',
                              string=str(p[i]))
        if len(relation) > 0:
            break
    # Check if the text says '_____ was born'
    first_and_last = ' '.join(name.split()[:2])
    last = name.split()[1]
    first = name.split()[0]
    name_was_born = []
    first_was_born = []
    last_was_born = []
    for i in range_look:
        name_was_born = re.findall(pattern = r'{0} was born'.format(first_and_last), string=str(p[i]) )
        last_was_born = re.findall(pattern = r'{0} was born'.format(last), string=str(p[i]))
        first_was_born = re.findall(pattern = r'{0} was born'.format(first), string=str(p[i]))
        if (len(name_was_born) > 0) or (len(last_was_born) > 0) or (len(first_was_born) > 0):
            break
    
    # If their name was born and they have at least 1 years mentioned, they are a person.
    if (len(name_was_born) > 0) and (len(temp) >= 1):
        print('PERSON FOUND' ,name)
        return True

    # If their last name was born and they have at least 1 years mentioned, they are a person.
    elif (len(last_was_born) > 0) and (len(temp) >= 1):
        print('PERSON FOUND' ,name)
        return True
    
    # If their first name was born and they have at least 1 years mentioned, they are a person.
    elif (len(last_was_born) > 0) and (len(temp) >= 1):
        print('PERSON FOUND' ,name)
        return True

    # Else check if the max and min dates would mean they are an adult (20 - 100)
    elif len(temp) > 0:
        years = [int(x) for x in temp]
        age_death = max(years) - min(years)
        if( age_death > 20) and (age_death < 100) and len(relation) > 0:
                print('PERSON FOUND' ,name)
                return True
        else:
            print('NOT A PERSON', name)
            return False
    else:
        print('NOT A PERSON', name)
        return False

In [40]:
def is_a_person(name):
    # Get page and parse
    #print('CHECKING', name)
    api_call = "http://en.wikipedia.org/w/api.php?action=parse&format=json&page="
    
    page = requests.get(api_call + name)
    source = bs(page.json()['parse']['text']['*'], 'html.parser')
    
    # Get all paragraph text
    p = source.find_all('p')
    
    # Check if there are year(s) in the first 2 p (birth/death date)
    temp = []
    for i in [0,1]:
        temp = re.findall(pattern= r'([\d]{4})', string=str(p[i]))
        if len(temp) > 0: 
            break

    # Check if there is a mention of a relationship in the first 6 paragraphs
    relation = []
    range_look = range(6)
    if len(p) < 6:
        range_look = range(len(p))
    for i in range_look:
        relation = re.findall(pattern= r'((?:Born|born|Father|father|mother|Mother|Son|son|Daugher|daughter|child))',
                              string=str(p[i]))
        if len(relation) > 0:
            break
    # Check if the text says '_____ was born'
    first_and_last = ' '.join(name.split()[:2])
    last = name.split()[1]
    first = name.split()[0]
    name_was_born = []
    first_was_born = []
    last_was_born = []
    for i in range_look:
        name_was_born = re.findall(pattern = r'{0} was born'.format(first_and_last), string=str(p[i]) )
        last_was_born = re.findall(pattern = r'{0} was born'.format(last), string=str(p[i]))
        first_was_born = re.findall(pattern = r'{0} was born'.format(first), string=str(p[i]))
        if (len(name_was_born) > 0) or (len(last_was_born) > 0) or (len(first_was_born) > 0):
            break
    
    # If their name was born and they have at least 1 years mentioned, they are a person.
    if (len(name_was_born) > 0) and (len(temp) >= 1):
        print('PERSON FOUND' ,name)
        return True

    # If their last name was born and they have at least 1 years mentioned, they are a person.
    elif (len(last_was_born) > 0) and (len(temp) >= 1):
        print('PERSON FOUND' ,name)
        return True
    
    # If their first name was born and they have at least 1 years mentioned, they are a person.
    elif (len(last_was_born) > 0) and (len(temp) >= 1):
        print('PERSON FOUND' ,name)
        return True

    # Else check if the max and min dates would mean they are an adult (20 - 100)
    elif len(temp) > 0:
        years = [int(x) for x in temp]
        age_death = max(years) - min(years)
        if( age_death > 20) and (age_death < 100) and len(relation) > 0:
                print('PERSON FOUND' ,name)
                return True
        else:
            print('NOT A PERSON', name)
            return False
    else:
        print('NOT A PERSON', name)
        return False


def add_persons_connections(G, name, source, approved, blacklist):
    front = 'https://en.wikipedia.org'

    # find all links
    p = source.find_all('p')

    links = []
    for para in p:
        p_source = bs(str(para), 'html.parser')
        lis = p_source.find_all('a', href = True)
        for lin in lis:
            links.append(lin)
    links = list(set(links))
    
    
    
    names = []
    
    # Loop over links
    for i in range(len(links)):
        
        # Get their name
        title = links[i].get('title')
        
        # If their link is already in the approved list, add the name
        if title in approved:
            names.append(title)
        # If their name is NOT IN the blacklist (meaning we have't checked it before) check
        elif title not in blacklist:
            try:
                doc = nlp(title)
                # IF they are classified as a person, check
                if doc.ents[0].label_ == 'PER':
                    
                    # Check if they are a person
                    if is_a_person(title) is True:
                        names.append(title)
                        approved.append(title)
                        time.sleep(.1)
                    else: # It isn't a person
                        blacklist.append(title)
                        time.sleep(.1)
                        continue
            except:
                continue
    # Unique names only
    their_unq_names = set(names)
    
    # Make tuples of (Person, Their connection)
    connections = [(name, x.split('(')[0]) for x in their_unq_names]
    
    # Add to graph
    G.add_edges_from(connections)
    # Add them and B Franklin
    G.add_edge('Benjamin Franklin', name)
    
    return G, approved, blacklist

In [41]:
%%time

# Make graph, approved list, and blacklist
G = nx.DiGraph()
approved = ['Benjamin Franklin']
blacklist = []
done = ['Benjamin Franklin']



def parse_persons_links()
for pos, name in enumerate(unq_names):
    try:
        # Get page
        api_call = "http://en.wikipedia.org/w/api.php?action=parse&format=json&page="

        page = requests.get(api_call + name[0])
        source = bs(page.json()['parse']['text']['*'], 'html.parser')

        # If name is not in approved and not in blacklist (haven't seen it before)
        if (name[0] not in approved) and (name[0] not in blacklist) and (name[0] not in done):
            print('NEW')
            if is_a_person(name[0]) is True:
                approved.append(name[0])
                G, approved, blacklist = add_persons_connections(G, name[0], source, approved, blacklist)
                done.append(name[0])
                time.sleep(1)
                print(name, len(unq_names) - pos)

            else:
                blacklist.append(name[0])
                print('REJECTED', name, len(unq_names) - pos)
        # Name IS in approved, but not in blacklist, just add connections       
        elif (name[0] not in blacklist) and (name[0] not in done):
            print('REPEAT')
            G, approved, blacklist = add_persons_connections(G, name[0], source, approved, blacklist)
            done.append(name[0])
            time.sleep(1)
            print(name, len(unq_names) - pos)
        else:
            continue
    except:
        print('FALIED:', name, len(unq_names) - pos)

NEW
PERSON FOUND Andrew Kippis
PERSON FOUND Philip Doddridge
NOT A PERSON Dissenting academies
NOT A PERSON Third voyage of James Cook
NOT A PERSON First voyage of James Cook
PERSON FOUND Nathaniel Lardner
PERSON FOUND George III of the United Kingdom
NOT A PERSON Second voyage of James Cook
PERSON FOUND Joseph Towers
NOT A PERSON English Dissenters
NOT A PERSON James Cook
('Andrew Kippis', 'https://en.wikipedia.org/wiki/Andrew_Kippis') 156
NEW
NOT A PERSON Catholic Church
REJECTED ('Catholic Church', 'https://en.wikipedia.org/wiki/Catholic_Church') 155
NEW
FALIED: ('Peter Foulger', 'https://en.wikipedia.org/wiki/Peter_Foulger') 154
NEW
PERSON FOUND Richard Bache
PERSON FOUND Richard Bache Jr.
PERSON FOUND Benjamin Franklin Bache (journalist)
PERSON FOUND Sarah Franklin Bache
PERSON FOUND Benjamin Franklin Bache
PERSON FOUND John Adams
PERSON FOUND Alexander Dallas Bache
NOT A PERSON George Washington
PERSON FOUND Deborah Read
PERSON FOUND Louis Franklin Bache
('Richard Bache', 'https:

('Jacques-Donatien Le Ray de Chaumont', 'https://en.wikipedia.org/wiki/Jacques-Donatien_Le_Ray_de_Chaumont') 141
REPEAT
PERSON FOUND Samuel Rowland Fisher
PERSON FOUND William Franklin
('Sarah Franklin Bache', 'https://en.wikipedia.org/wiki/Sarah_Franklin_Bache') 140
NEW
PERSON FOUND John Hadley (chemist)
('John Hadley (chemist)', 'https://en.wikipedia.org/wiki/John_Hadley_(chemist)') 139
NEW
PERSON FOUND William Hewson (surgeon)
PERSON FOUND Paul Knapman
NOT A PERSON Benjamin Franklin House
PERSON FOUND William Hunter (anatomist)
NOT A PERSON Cell membrane
('William Hewson (surgeon)', 'https://en.wikipedia.org/wiki/William_Hewson_(surgeon)') 138
NEW
FALIED: ('John Dickinson (Pennsylvania and Delaware)', 'https://en.wikipedia.org/wiki/John_Dickinson_(Pennsylvania_and_Delaware)') 137
NEW
PERSON FOUND Henry Steele Commager
PERSON FOUND Herbert Agar
PERSON FOUND Theodore Parker
PERSON FOUND Adlai Stevenson II
PERSON FOUND Albert Bushnell Hart
PERSON FOUND Samuel Eliot Morison
PERSON FOUND

REJECTED ("Benjamin Franklin's phonetic alphabet", 'https://en.wikipedia.org/wiki/Benjamin_Franklin%27s_phonetic_alphabet') 116
NEW
NOT A PERSON Old Slaughter's Coffee House
REJECTED ("Old Slaughter's Coffee House", 'https://en.wikipedia.org/wiki/Old_Slaughter%27s_Coffee_House') 115
NEW
FALIED: ('Bequest', 'https://en.wikipedia.org/wiki/Bequest') 114
NEW
PERSON FOUND James Franklin (printer)
PERSON FOUND Ann Smith Franklin
NOT A PERSON Poor Robin
('James Franklin (printer)', 'https://en.wikipedia.org/wiki/James_Franklin_(printer)') 113
REPEAT
NOT A PERSON Fleet Prison
PERSON FOUND Andrew Bradford
PERSON FOUND Daniel Defoe
('Samuel Keimer', 'https://en.wikipedia.org/wiki/Samuel_Keimer') 112
NEW
NOT A PERSON Louis Timothee
REJECTED ('Louis Timothee', 'https://en.wikipedia.org/wiki/Louis_Timothee') 111
NEW
PERSON FOUND Archibald Spencer
PERSON FOUND Pieter van Musschenbroek
PERSON FOUND Ebenezer Kinnersley
NOT A PERSON Leo Lemay
PERSON FOUND Isaac Greenwood
PERSON FOUND Lewis Evans (surve

NOT A PERSON Habsburg Monarchy
PERSON FOUND Étienne Clavière
NOT A PERSON Joachim von Blumenthal
('Honoré Gabriel Riqueti, comte de Mirabeau', 'https://en.wikipedia.org/wiki/Honor%C3%A9_Gabriel_Riqueti,_comte_de_Mirabeau') 92
NEW
NOT A PERSON Benjamin Franklin in popular culture
REJECTED ('Benjamin Franklin in popular culture', 'https://en.wikipedia.org/wiki/Benjamin_Franklin_in_popular_culture') 89
REPEAT
PERSON FOUND Blaise Pascal
PERSON FOUND André-Hercule de Fleury
NOT A PERSON Charles VI, Holy Roman Emperor
PERSON FOUND Jean-Baptiste Oudry
PERSON FOUND Frederick William I of Prussia
PERSON FOUND Victor Amadeus II of Sardinia
PERSON FOUND Philip V of Spain
PERSON FOUND Marc-Pierre de Voyer de Paulmy d'Argenson
PERSON FOUND Marie Anne de Mailly
NOT A PERSON Jacques Claude Marie Vincent de Gournay
PERSON FOUND Mariana Victoria of Spain
PERSON FOUND Louise Julie de Mailly
PERSON FOUND Emmanuel Le Roy Ladurie
NOT A PERSON Peter the Great
PERSON FOUND François Couperin
PERSON FOUND Pete

PERSON FOUND Daniel Leonard
PERSON FOUND Thomas Paine
NOT A PERSON Gordon S. Wood
PERSON FOUND James Iredell
NOT A PERSON States' rights
PERSON FOUND Charles Maurice de Talleyrand-Périgord
PERSON FOUND Thomas Preston (British Army officer)
PERSON FOUND William Stephens Smith
PERSON FOUND Elbridge Gerry
NOT A PERSON John Adams Birthplace
PERSON FOUND Edmund Randolph
PERSON FOUND Joseph Galloway
NOT A PERSON Aaron Burr
PERSON FOUND Charles Adams (1770–1800)
NOT A PERSON First inauguration of Thomas Jefferson
PERSON FOUND Oliver Ellsworth
PERSON FOUND Abigail Adams Smith
PERSON FOUND Matthew Lyon
PERSON FOUND Joseph Mayhew
NOT A PERSON Adams political family
PERSON FOUND Arthur Lee (diplomat)
PERSON FOUND Ron Chernow
NOT A PERSON Civic virtue
PERSON FOUND John Fries
PERSON FOUND Friedrich Wilhelm von Thulemeyer
PERSON FOUND John Marshall
PERSON FOUND Sally Hemings
PERSON FOUND Thomas Boylston Adams (1772–1832)
PERSON FOUND James T. Callender
PERSON FOUND Henry Laurens
NOT A PERSON John Ad

('Thomas Young (scientist)', 'https://en.wikipedia.org/wiki/Thomas_Young_(scientist)') 58
NEW
FALIED: ('Gout', 'https://en.wikipedia.org/wiki/Gout') 57
REPEAT
PERSON FOUND Peter Wedderburn, Lord Chesterhall
PERSON FOUND James St Clair-Erskine, 2nd Earl of Rosslyn
PERSON FOUND Charles Macklin
NOT A PERSON Ayr Burghs (UK Parliament constituency)
PERSON FOUND Thellusson v Woodford
PERSON FOUND Henry Addington, 1st Viscount Sidmouth
NOT A PERSON East Lothian
PERSON FOUND Thomas Sheridan (actor)
NOT A PERSON John Wilkes
PERSON FOUND Charles James Fox
('Alexander Wedderburn, 1st Earl of Rosslyn', 'https://en.wikipedia.org/wiki/Alexander_Wedderburn,_1st_Earl_of_Rosslyn') 56
NEW
FALIED: ('Banknote', 'https://en.wikipedia.org/wiki/Banknote') 55
NEW
FALIED: ("Gentleman's club", 'https://en.wikipedia.org/wiki/Gentleman%27s_club') 54
NEW
FALIED: ('Bellows', 'https://en.wikipedia.org/wiki/Bellows') 53
NEW
PERSON FOUND Cotton Mather
NOT A PERSON Josiah Cotton
NOT A PERSON Hayy ibn Yaqdhan
NOT A PERS

REJECTED ('Thomas Percival', 'https://en.wikipedia.org/wiki/Thomas_Percival') 22
REPEAT
NOT A PERSON J. G. A. Pocock
NOT A PERSON Bunhill Fields
PERSON FOUND John Haygarth
NOT A PERSON Gravel Pit Chapel
PERSON FOUND John Balguy
PERSON FOUND John Eames
PERSON FOUND Ralph Cudworth
NOT A PERSON Old Jewry Meeting-house
NOT A PERSON Bayesian statistics
PERSON FOUND Thomas Jervis (minister)
PERSON FOUND Josiah Tucker
PERSON FOUND Charles Stanhope, 3rd Earl Stanhope
PERSON FOUND William Coxe (historian)
NOT A PERSON John Brand (political writer)
NOT A PERSON Lord Shelburne
PERSON FOUND George Lyttelton, 1st Baron Lyttelton
PERSON FOUND Samuel Clarke
PERSON FOUND Ezra Stiles
PERSON FOUND William Markham (bishop)
PERSON FOUND James Mackintosh
PERSON FOUND William Godwin
PERSON FOUND Joseph Butler
PERSON FOUND Dugald Stewart
PERSON FOUND James Burgh
PERSON FOUND John Howard (prison reformer)
PERSON FOUND Adam Ferguson
PERSON FOUND Thomas Bayes
PERSON FOUND John Howlett (political economist)
PERS

PERSON FOUND Edward Montagu, 2nd Earl of Manchester
NOT A PERSON Wenceslaus Hollar
PERSON FOUND Peter Paul Rubens
PERSON FOUND Oliver St John
PERSON FOUND Paolo Veronese
PERSON FOUND Jane Seymour
PERSON FOUND Elizabeth I of England
NOT A PERSON Dunfermline Palace
PERSON FOUND Henry VI of England
NOT A PERSON Chapel Royal
PERSON FOUND Henry Vane the Elder
PERSON FOUND Gerard van Honthorst
PERSON FOUND Society of King Charles the Martyr
PERSON FOUND Robert Carey, 1st Earl of Monmouth
PERSON FOUND William Prynne
('Charles I of England', 'https://en.wikipedia.org/wiki/Charles_I_of_England') 5
REPEAT
NOT A PERSON Establishment Clause
NOT A PERSON Lee Resolution
PERSON FOUND George Hammond (diplomat)
PERSON FOUND Dabney Carr
PERSON FOUND George Logan
PERSON FOUND George Clinton (vice president)
PERSON FOUND Francis Fauquier
PERSON FOUND Thomas Jefferson Randolph
NOT A PERSON Virginia Statute for Religious Freedom
PERSON FOUND Black Hoof
PERSON FOUND Anthony Merry
PERSON FOUND James Hemings
P

# Network Analysis: Descriptives

### Number of Nodes (People in Network)

In [210]:
len(G.nodes())

1066

### Number of Edges (Connections)

In [211]:
len(G.edges())

1400

### Is the network strongly connected? (Basically, is there a path from any node to any other node?)

In [55]:
nx.is_strongly_connected(G)

False

No, and this makes sense. If it was strongly connected, then it would mean that one every single page, there would be either a link back to the page that linked to it, or a link to someone else/others in the network. This is unlikely for shorter pages with only a few links.

### Is the network weakly connected? (Basically, are all the nodes linked or are there more than one connected groups?)

In [56]:
nx.is_weakly_connected(G)

True

Yes, and this also makes sense. All pages are either links from Benjamin Franklin or a link from a link from Benjamin Franklin, so they must all be connected. At no point could we have nodes that aren't a part of the main network.

## Let's convert our graph to an undirected network for other analyses.

In [234]:
G_un = G.to_undirected()

### We'll also drop anyone who only has one connection, so that we have a simpler network and only keep people who are connected to more than one person.

In [238]:
di = dict(nx.degree(G_un))

degree = []
for i in di:
    if di[i] == 1:
        degree.append(i)
G_un.remove_nodes_from(degree)

### What node(s) are at the center of the network?

In [237]:
nx.center(G_un)

['Benjamin Franklin']

Not suprising here, because all nodes have to link back in 2 or less edges to only Benjamin Franklin. This is his network!

### What nodes are on the periphery of the network? 

In [239]:
len(nx.periphery(G_un))

140

In [232]:
100 - (((len(G_un) - len(nx.periphery(G_un))) / len(G_un)) * 100)

63.92694063926941

In [246]:
nx.diameter(G_un)

4

64% of the network is on the periphery. This makes sense as the diameter is only 4, and many links have to go 2 to Benjamin and then 2 to other nodes.

## Who are the top 20 most connected people?

In [248]:
di = dict(nx.degree(G_un))

degree = []
for i in di:
    degree.append((i, di[i]))
degree.sort(key = lambda x: x[1], reverse = True)

In [247]:
degree[:20]

[('Benjamin Franklin', 69),
 ('John Adams', 37),
 ('Thomas Jefferson', 35),
 ('Joseph Priestley', 29),
 ('Richard Price', 24),
 ('George III of the United Kingdom', 22),
 ('Adam Smith', 18),
 ('Matthew Boulton', 17),
 ('Jonathan Swift', 16),
 ('Louis XVI of France', 16),
 ('Alexander Pope', 15),
 ('Louis XV of France', 14),
 ('Thomas Hutchinson (governor)', 14),
 ('Leonhard Euler', 13),
 ('Erasmus Darwin', 12),
 ('Alexander Wedderburn, 1st Earl of Rosslyn', 11),
 ('William Penn', 11),
 ('James Watt', 11),
 ('John Hancock', 11),
 ('Joseph Addison', 10)]

# Graph of Full Network

I chose Fruchterman-Reingold algorithm because it provided a clearer view of the clusters of people linked to more central nodes.

In [222]:
graph_3d(G, 'BFrank')

## Graph of Only Degree >1 (More than 1 connection)

In [221]:
graph_3d(G_un, 'BenFrank')

In [215]:
import igraph as ig
import plotly.plotly as py
import plotly.graph_objs as go
import plotly

temp = pd.read_csv('Secret\cred.txt', header = None)

plotly.tools.set_credentials_file(username=temp[0][0].strip(), api_key=temp[1][0].strip())

In [219]:
def graph_3d(G_un, filename):    
    nodes_dict = {}
    for i, v in enumerate(list(G_un.nodes())):
        nodes_dict[v] = i

    Edges = [(nodes_dict[x[0]], nodes_dict[x[1]]) for x in list(G_un.edges())]

    G_ig_un = ig.Graph(Edges, directed = False)

    layt=G_ig_un.layout('fr_3d', dim=3)

    labels= list(G_un.nodes())
    labels = [str(x) for x in labels]

    group = []

    for i in dict(nx.degree(G_un)):
        group.append(dict(nx.degree(G_un))[i])

    N = len(labels)

    Xn=[layt[k][0] for k in range(N)]# x-coordinates of nodes
    Yn=[layt[k][1] for k in range(N)]# y-coordinates
    Zn=[layt[k][2] for k in range(N)]# z-coordinates
    Xe=[]
    Ye=[]
    Ze=[]
    for e in Edges:
        Xe+=[layt[e[0]][0],layt[e[1]][0], None]# x-coordinates of edge ends
        Ye+=[layt[e[0]][1],layt[e[1]][1], None]
        Ze+=[layt[e[0]][2],layt[e[1]][2], None]

    import plotly.plotly as py
    import plotly.graph_objs as go

    trace1=go.Scatter3d(x=Xe,
                   y=Ye,
                   z=Ze,
                   mode='lines',
                   line=dict(color='rgb(125,125,125)', width=1),
                   hoverinfo='none'
                   )
    trace2=go.Scatter3d(x=Xn,
                   y=Yn,
                   z=Zn,
                   mode='markers',
                   name='actors',
                   marker=dict(symbol='circle',
                                 size=[x+6 for x in group],
                                 color=group,
                                 colorscale='Bluered',
                                 line=dict(color='rgb(50,50,50)', width=0.5)
                                 ),
                   text=labels,
                   hoverinfo='text'
                   )
    axis=dict(showbackground=False,
              showline=False,
              zeroline=False,
              showgrid=False,
              showticklabels=False,
              title=''
              )
    layout = go.Layout(
             title="Benjamin Franklin's Wikipedia Network (3D visualization)",
             width=1000,
             height=1000,
             showlegend=False,
             scene=dict(
                 xaxis=dict(axis),
                 yaxis=dict(axis),
                 zaxis=dict(axis),
            ),
         margin=dict(
            t=100
        ),
        hovermode='closest',
        annotations=[
               dict(
               showarrow=False,
                text="Data source: Wikipedia",
                xref='paper',
                yref='paper',
                x=0,
                y=0.1,
                xanchor='left',
                yanchor='bottom',
                font=dict(
                size=14
                )
                )
            ],    )
    data=[trace1, trace2]
    fig=go.Figure(data=data, layout=layout)

    return py.iplot(fig, filename = filename, auto_open = True, world_readable = True)

## SAVING NETWORKX GRAPHS

In [49]:
nx.write_adjlist(G, 'G_directed_v1.txt')
nx.write_gpickle(G, 'G_dir_v1.P')

In [50]:
G_temp = nx.read_adjlist('G_directed_v1.txt')
len(G_temp.nodes()), len(G_temp.edges())

(1482, 4141)

In [98]:
G_temp2 = nx.read_gpickle('G_dir_v1.P')
len(G_temp2.nodes()), len(G_temp2.edges())

(1066, 1400)

Loop through and identify names. If it is a name of a person, then store the link and the name of the person as a network link.
